In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import cmath
import math
import time
import pandas as pd

### We fix the parameters

In [192]:
# Fixed Parameters
S0 = 1900
K = 2000
k = np.log(K)
r = 0.02
q = 0.0187


In [184]:
# Parameters for FFT 

n = 9
N = 2**n



#step-size
eta = 0.25
# damping factor
alpha = 0.4

# integration parameter
N2 = 1000


In [185]:
# step-size in log strike space
lda = (2*math.pi/N)/eta;

#Choice of beta
#beta = np.log(S0)-N*lda/2
beta = np.log(K)

In [186]:
#model-specific Parameters
model = 'GBM'

In [187]:
params = []     
if (model == 'GBM'):
    
    sig = 0.36
    params.append(sig);

### Characteristic function of the log stock price

In [188]:
def generic_CF(u, params, S0, r, q, T, model):
    
    if (model == 'GBM'):
        
        sig = params[0]
        mu = np.log(S0) + (r-q-sig**2/2)*T
        a = sig*np.sqrt(T)
        phi = np.exp(1j*mu*u-(a*u)**2/2)
        
    return phi

### Direct evaluation of the integral

In [8]:
def evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N, model):
    
    # Just one strike at a time
    # no need for Fast Fourier Transform
    
    # discount factor
    df = math.exp(-r*T)
    
    sum1 = 0
    for j in range(N):
        nuJ = j*eta
        psi_nuJ = df*generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        sum1 += np.exp(-1j*nuJ*k)*psi_nuJ*wJ
        
    cT_k = (np.exp(-alpha*k)/math.pi)*sum1
    
    return np.real(cT_k) 

## FFT

In [9]:
def genericFFT(params, S0, K, r, q, T, alpha, eta, n, model):
    
    N = 2**n
    
    # step-size in log strike space
    lda = (2*np.pi/N)/eta
    
    #Choice of beta
    beta = np.log(K)-N*lda/2
    #beta = np.log(K)
    
    # forming vector x and strikes km for m=1,...,N
    km = np.zeros((N))
    xX = np.zeros((N))
    
    # discount factor
    df = math.exp(-r*T)
    
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
    
    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        xX[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ
     
    yY = np.fft.fft(xX)
    cT_km = np.zeros((N))  
    for i in range(N):
        multiplier = np.exp(-alpha*km[i])/math.pi
        cT_km[i] = multiplier*np.real(yY[i])
    
    return km, cT_km


In [11]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
T = 0.25
    
print(' ')
start_time = time.time()
km, cT_km = genericFFT(params, S0, K, r, q, T, alpha, eta, n, model)
#cT_k = cT_km[0]
cT_k = np.interp(k, km, cT_km)


elapsed_time = time.time() - start_time
    
#cT_k = np.interp(np.log(), km, cT_km)
print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
#print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
print('FFT execution time was %0.7f' % elapsed_time)
    
print(' ')
start_time = time.time()
cT_k = evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N2, model)
elapsed_time = time.time() - start_time
print("Option via Integration: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
print('Evaluation of integral time was %0.7f' % elapsed_time)

 
Model is GBM
-------------------
 
Option via FFT: for strike 1999.9999999999998 the option premium is 95.3281
FFT execution time was 0.0069549
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.3281
Evaluation of integral time was 0.0102038


In [12]:
n_list = [9,11,13,15]
K_list = [2000, 2100, 2200]
k_list = np.log(K)
alpha_list = [0.4, 1.0, 1.4, 3.0]

In [13]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
T = 0.25
dict_prices = {}
for i in range(len(K_list)):
    list_prices_K = []
    for j in range(len(n_list)):
        list_prices_n = []
        for z in range (len(alpha_list)):
            
            K= K_list[i]
            n = n_list[j]
            k = np.log(K)
            alpha = alpha_list[z]
            print(' ')
            start_time = time.time()
            km, cT_km = genericFFT(params, S0, K, r, q, T, alpha, eta, n, model)
            #cT_k = cT_km[0]
            cT_k = np.interp(k, km, cT_km)
            list_prices_n.append(cT_k)


            elapsed_time = time.time() - start_time
    
            #cT_k = np.interp(np.log(), km, cT_km)
            print("Option via FFT: for n is "+str(n)+' with alpha = '+str(alpha))
            print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
            #print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
            print('FFT execution time was %0.7f' % elapsed_time)
    
            print(' ')
            start_time = time.time()
            cT_k = evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N2, model)
            elapsed_time = time.time() - start_time
            print("Option via Integration: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
            print('Evaluation of integral time was %0.7f' % elapsed_time)
        list_prices_K.append(list_prices_n)
    dict_prices[K_list[i]]=list_prices_K

 
Model is GBM
-------------------
 
Option via FFT: for n is 9 with alpha = 0.4
Option via FFT: for strike 1999.9999999999998 the option premium is 95.3281
FFT execution time was 0.0042338
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.3281
Evaluation of integral time was 0.0082381
 
Option via FFT: for n is 9 with alpha = 1.0
Option via FFT: for strike 1999.9999999999998 the option premium is 95.2467
FFT execution time was 0.0035570
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.2467
Evaluation of integral time was 0.0077162
 
Option via FFT: for n is 9 with alpha = 1.4
Option via FFT: for strike 1999.9999999999998 the option premium is 95.2467
FFT execution time was 0.0035057
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.2467
Evaluation of integral time was 0.0083778
 
Option via FFT: for n is 9 with alpha = 3.0
Option via FFT: for strike 1999.9999999999998 the option premium is 95

In [20]:
prices_FFT_K_2200 = pd.DataFrame(dict_prices[2200], columns = alpha_list)
prices_FFT_K_2200['N'] = n_list
prices_FFT_K_2200 = prices_FFT_K_2200.set_index('N')
prices_FFT_K_2200

,0.4,1.0,1.4,3.0
N,,,,
9,43.028597,42.947175,42.947175,42.947175
11,43.028597,42.947175,42.947175,42.947175
13,43.028597,42.947175,42.947175,42.947175
15,43.028597,42.947175,42.947175,42.947175


In [19]:
prices_FFT_K_2100 = pd.DataFrame(dict_prices[2100], columns = alpha_list)
prices_FFT_K_2100['N'] = n_list
prices_FFT_K_2100 = prices_FFT_K_2100.set_index('N')
prices_FFT_K_2100

,0.4,1.0,1.4,3.0
N,,,,
9,64.916042,64.83462,64.83462,64.83462
11,64.916042,64.83462,64.83462,64.83462
13,64.916042,64.83462,64.83462,64.83462
15,64.916042,64.83462,64.83462,64.83462


In [18]:
prices_FFT_K_2000 = pd.DataFrame(dict_prices[2000], columns = alpha_list)
prices_FFT_K_2000['N'] = n_list
prices_FFT_K_2000 = prices_FFT_K_2000.set_index('N')
prices_FFT_K_2000

,0.4,1.0,1.4,3.0
N,,,,
9,95.328114,95.246692,95.246692,95.246692
11,95.328114,95.246692,95.246692,95.246692
13,95.328114,95.246692,95.246692,95.246692
15,95.328114,95.246692,95.246692,95.246692


### FrFFT

### Part 1

In [64]:
def genericFrFFT(params, S0, K, r, q, T, alpha, eta, n, lda, model):
    
    N = 2**n
    gamma = eta*lda/(2*math.pi)

    #Choice of beta
    beta = np.log(K)-N*lda/2
    #beta = np.log(K)

    # initialize x, y, z, and cT_km
    km = np.zeros((N))
    x = np.zeros((N))
    y = np.zeros((2*N), dtype=np.complex)
    z = np.zeros((2*N), dtype=np.complex)
    cT_km = np.zeros((N)) 

    # discount factor
    df = math.exp(-r*T)

    # compute x
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))

    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        x[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ

    # set up y
    for i in range(N):
        y[i] = np.exp(-1j*math.pi*gamma*i**2)*x[i]
    y[N:] = 0

    # set up z
    for i in range(N):
        z[i] = np.exp(1j*math.pi*gamma*i**2)
    z[N:] = z[:N][::-1]

    # compute xi_hat
    xi_hat = np.fft.ifft(np.fft.fft(y) * np.fft.fft(z))

    # compute call prices
    for i in range(N):
        cT_km[i] = np.exp(-alpha*(beta + i*lda))/math.pi * (np.exp(-1j*math.pi*gamma*i**2)*xi_hat[i]).real

    return km, cT_km

In [189]:
n_Fr_list = [6,7,8,9]
K_list = [2000, 2100, 2200]
k_list = np.log(K)
alpha_list = [0.4, 1.0, 1.4, 3.0]
eta = 0.25
lda_FrFFT =0.1

In [67]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
T = 0.25
dict_prices = {}
for i in range(len(K_list)):
    list_prices_K = []
    for j in range(len(n_Fr_list)):
        list_prices_n = []
        for z in range (len(alpha_list)):
            
            K= K_list[i]
            n_FrFFT = n_Fr_list[j]
            k = np.log(K)
            alpha = alpha_list[z]
            # FrFFT
            print(' ')
            start_time = time.time()
            km, cT_km = genericFrFFT(params, S0, K, r, q, T, alpha, eta, n_FrFFT, lda_FrFFT, model)
            #cT_k = cT_km[0]
            cT_k = np.interp(k, km, cT_km)
            list_prices_n.append(cT_k)

            elapsed_time = time.time() - start_time
            
            print("Option via FrFFT: for n is "+str(n_FrFFT)+' with alpha = '+str(alpha))
            #cT_k = np.interp(np.log(), km, cT_km)
            print("Option via FrFFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
            #print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
            print('FrFFT execution time was %0.7f' % elapsed_time)
    
            print(' ')
            start_time = time.time()
            cT_k = evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N2, model)
            elapsed_time = time.time() - start_time
            print("Option via Integration: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
            print('Evaluation of integral time was %0.7f' % elapsed_time)
        list_prices_K.append(list_prices_n)
    dict_prices[K_list[i]]=list_prices_K

 
Model is GBM
-------------------
 
Option via FrFFT: for n is 6 with alpha = 0.4
Option via FrFFT: for strike 1999.9999999999998 the option premium is 120.7730
FrFFT execution time was 0.0012980
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.3281
Evaluation of integral time was 0.0096519
 
Option via FrFFT: for n is 6 with alpha = 1.0
Option via FrFFT: for strike 1999.9999999999998 the option premium is 49.1717
FrFFT execution time was 0.0012560
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.2467
Evaluation of integral time was 0.0086141
 
Option via FrFFT: for n is 6 with alpha = 1.4
Option via FrFFT: for strike 1999.9999999999998 the option premium is 47.6224
FrFFT execution time was 0.0009379
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.2467
Evaluation of integral time was 0.0089698
 
Option via FrFFT: for n is 6 with alpha = 3.0
Option via FrFFT: for strike 1999.9999999999998 t

In [68]:
prices_Fr_FFT_K_2200 = pd.DataFrame(dict_prices[2200], columns = alpha_list)
prices_Fr_FFT_K_2200['N'] = n_Fr_list
prices_Fr_FFT_K_2200 = prices_Fr_FFT_K_2200.set_index('N')
prices_Fr_FFT_K_2200

,0.4,1.0,1.4,3.0
N,,,,
6,94.898345,23.389593,21.877956,21.461534
7,27.167768,21.479100,21.476479,21.474552
8,238.692187,185.529288,157.561692,85.910618
9,376.380718,224.016539,160.845355,52.720385


In [69]:
prices_Fr_FFT_K_2100 = pd.DataFrame(dict_prices[2100], columns = alpha_list)
prices_Fr_FFT_K_2100['N'] = n_Fr_list
prices_Fr_FFT_K_2100 = prices_Fr_FFT_K_2100.set_index('N')
prices_Fr_FFT_K_2100

,0.4,1.0,1.4,3.0
N,,,,
6,105.764624,34.200661,32.662995,32.201483
7,38.112215,32.422479,32.419211,32.415878
8,274.603822,215.336651,184.153252,104.263512
9,400.770596,242.635371,177.070396,64.848165


In [70]:
prices_Fr_FFT_K_2000 = pd.DataFrame(dict_prices[2000], columns = alpha_list)
prices_Fr_FFT_K_2000['N'] = n_Fr_list
prices_Fr_FFT_K_2000 = prices_Fr_FFT_K_2000.set_index('N')
prices_Fr_FFT_K_2000

,0.4,1.0,1.4,3.0
N,,,,
6,120.772992,49.171706,47.622351,47.189255
7,53.316499,47.626015,47.622463,47.619876
8,315.188499,249.716638,215.264536,127.000603
9,429.422837,265.516546,197.557769,81.238264


### Part 2: we change the values of lambda and beta

In [212]:
def genericFrFFT(params, S0, K, r, q, T, alpha, eta, n, lda, model):
    
    N = 2**n
    gamma = eta*lda/(2*math.pi)

    #Choice of beta
    beta = np.log(K)
    #beta = np.log(K)

    # initialize x, y, z, and cT_km
    km = np.zeros((N))
    x = np.zeros((N))
    y = np.zeros((2*N), dtype=np.complex)
    z = np.zeros((2*N), dtype=np.complex)
    cT_km = np.zeros((N)) 

    # discount factor
    df = math.exp(-r*T)

    # compute x
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))

    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        x[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ

    # set up y
    for i in range(N):
        y[i] = np.exp(-1j*math.pi*gamma*i**2)*x[i]
    y[N:] = 0

    # set up z
    for i in range(N):
        z[i] = np.exp(1j*math.pi*gamma*i**2)
    z[N:] = z[:N][::-1]

    # compute xi_hat
    xi_hat = np.fft.ifft(np.fft.fft(y) * np.fft.fft(z))

    # compute call prices
    for i in range(N):
        cT_km[i] = np.exp(-alpha*(beta + i*lda))/math.pi * (np.exp(-1j*math.pi*gamma*i**2)*xi_hat[i]).real

    return km, cT_km

In [213]:
n_Fr_list = [6,7,8,9]
K_list = [2000, 2100, 2200]
k_list = np.log(K)
alpha_list = [0.4, 1.0, 1.4, 3.0]
eta = 0.25
lda_FrFFT =0.0001

In [214]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
T = 0.25
dict_prices = {}
for i in range(len(K_list)):
    list_prices_K = []
    for j in range(len(n_Fr_list)):
        list_prices_n = []
        for z in range (len(alpha_list)):
            
            K= K_list[i]
            n_FrFFT = n_Fr_list[j]
            k = np.log(K)
            alpha = alpha_list[z]
            # FrFFT
            print(' ')
            start_time = time.time()
            km, cT_km = genericFrFFT(params, S0, K, r, q, T, alpha, eta, n_FrFFT, lda_FrFFT, model)
            #cT_k = cT_km[0]
            cT_k = np.interp(k, km, cT_km)
            list_prices_n.append(cT_k)

            elapsed_time = time.time() - start_time
            
            print("Option via FrFFT: for n is "+str(n_FrFFT)+' with alpha = '+str(alpha))
            #cT_k = np.interp(np.log(), km, cT_km)
            print("Option via FrFFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
            #print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
            print('FrFFT execution time was %0.7f' % elapsed_time)
    
            print(' ')
            start_time = time.time()
            cT_k = evaluateIntegral(params, S0, K, r, q, T, alpha, eta, N2, model)
            elapsed_time = time.time() - start_time
            print("Option via Integration: for strike %s the option premium is %6.4f" % (np.exp(k), cT_k))
            print('Evaluation of integral time was %0.7f' % elapsed_time)
        list_prices_K.append(list_prices_n)
    dict_prices[K_list[i]]=list_prices_K

 
Model is GBM
-------------------
 
Option via FrFFT: for n is 6 with alpha = 0.4
Option via FrFFT: for strike 1999.9999999999998 the option premium is 95.3845
FrFFT execution time was 0.0014420
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.3281
Evaluation of integral time was 0.0101540
 
Option via FrFFT: for n is 6 with alpha = 1.0
Option via FrFFT: for strike 1999.9999999999998 the option premium is 95.3041
FrFFT execution time was 0.0009530
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.2467
Evaluation of integral time was 0.0103288
 
Option via FrFFT: for n is 6 with alpha = 1.4
Option via FrFFT: for strike 1999.9999999999998 the option premium is 95.2998
FrFFT execution time was 0.0012619
 
Option via Integration: for strike 1999.9999999999998 the option premium is 95.2467
Evaluation of integral time was 0.0099089
 
Option via FrFFT: for n is 6 with alpha = 3.0
Option via FrFFT: for strike 1999.9999999999998 th

In [215]:
prices_Fr_FFT_K_2200 = pd.DataFrame(dict_prices[2200], columns = alpha_list)
prices_Fr_FFT_K_2200['N'] = n_Fr_list
prices_Fr_FFT_K_2200 = prices_Fr_FFT_K_2200.set_index('N')
prices_Fr_FFT_K_2200

,0.4,1.0,1.4,3.0
N,,,,
6,43.009624,42.950147,42.963261,42.991135
7,43.028608,42.947185,42.947184,42.947179
8,43.028608,42.947185,42.947184,42.947179
9,43.028608,42.947185,42.947184,42.947179


In [216]:
prices_Fr_FFT_K_2100 = pd.DataFrame(dict_prices[2100], columns = alpha_list)
prices_Fr_FFT_K_2100['N'] = n_Fr_list
prices_Fr_FFT_K_2100 = prices_Fr_FFT_K_2100.set_index('N')
prices_Fr_FFT_K_2100

,0.4,1.0,1.4,3.0
N,,,,
6,64.941191,64.876301,64.883298,64.875930
7,64.916055,64.834631,64.834629,64.834623
8,64.916055,64.834631,64.834629,64.834623
9,64.916055,64.834631,64.834629,64.834623


In [217]:
prices_Fr_FFT_K_2000 = pd.DataFrame(dict_prices[2000], columns = alpha_list)
prices_Fr_FFT_K_2000['N'] = n_Fr_list
prices_Fr_FFT_K_2000 = prices_Fr_FFT_K_2000.set_index('N')
prices_Fr_FFT_K_2000

,0.4,1.0,1.4,3.0
N,,,,
6,95.384535,95.304088,95.299808,95.252036
7,95.328127,95.246703,95.246700,95.246693
8,95.328127,95.246702,95.246700,95.246693
9,95.328127,95.246702,95.246700,95.246693


### COS METHOD

In [71]:
def generic_CF(u, params, S0, r, q, T, model):
    
    if (model == 'GBM'):
        
        sig = params[0]
        mu = np.log(S0) + (r-q-sig**2/2)*T
        a = sig*np.sqrt(T)
        phi = np.exp(1j*mu*u-(a*u)**2/2)
        
    return phi

In [98]:
from scipy import integrate
import numpy as np

In [159]:
def Price_Option(N,a,b,params, S0, r, q, T, model,K):
    i = np.complex(0.0,1.0) #assigning i=sqrt(-1)
    k = np.linspace(0,N-1,N)
    u = np.zeros([1,N])
    u = k * np.pi / (b-a)
    
    C = np.exp(-r*T)
    
    #A_k coefficients
    F_k    = np.real(generic_CF(u,params, S0/K, r, q, T, model) * np.exp(-i * u * a));
    F_k[0] = F_k[0] * 0.5; # adjustment for the first term
    
    #V_k coefficients
    V_k = np.zeros(N)
    
    for j in range(N):
        fonction = lambda y: max((np.exp(y)-1),0)*np.cos(u[j]*(y-a))
        V_k[j] = K*integrate.quad(fonction, 0, b)[0]*2*(1/(b-a))
    
    return(C*(F_k@V_k))

In [169]:
K_list = [2000,2100,2200]
N_list = [2**10]
interval_list = [[-1,1],[-4,4],[-8,8],[-12,12],[-0.5,0.5],[-0.01,0.01],[-16,16],[-32,32]]

In [170]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
T = 0.25
dict_prices = {}
for i in range(len(K_list)):
    list_prices_K = []
    for j in range(len(N_list)):
        list_prices_n = []
        for z in range (len(interval_list)):
            
            K= K_list[i]
            N_value = N_list[j]
            a = interval_list[z][0]
            b = interval_list[z][1]
            # COS
            print(' ')
            start_time = time.time()
            price = Price_Option(N_value,a,b,params, S0, r, q, T, model,K)
            list_prices_n.append(price)

            elapsed_time = time.time() - start_time
            
            print("Option via COS: for N is "+str(N_value)+' with interval = ['+str(a)+', '+str(b)+']')
            print("Option via COS: for strike %s the option premium is %6.4f" % (K, price))
            #print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
            print('COS execution time was %0.7f' % elapsed_time)
            
        list_prices_K.append(list_prices_n)
    dict_prices[K_list[i]]=list_prices_K

 
Model is GBM
-------------------
 
Option via COS: for N is 1024 with interval = [-1, 1]
Option via COS: for strike 2000 the option premium is 95.2467
COS execution time was 3.9837127
 
Option via COS: for N is 1024 with interval = [-4, 4]
Option via COS: for strike 2000 the option premium is 95.2467
COS execution time was 4.0911031
 
Option via COS: for N is 1024 with interval = [-8, 8]
Option via COS: for strike 2000 the option premium is 95.2467
COS execution time was 4.1525679
 
Option via COS: for N is 1024 with interval = [-12, 12]
Option via COS: for strike 2000 the option premium is 95.2467
COS execution time was 4.0920768
 
Option via COS: for N is 1024 with interval = [-0.5, 0.5]
Option via COS: for strike 2000 the option premium is 94.9840
COS execution time was 3.8998940
 
Option via COS: for N is 1024 with interval = [-0.01, 0.01]
Option via COS: for strike 2000 the option premium is 4.9917
COS execution time was 3.5012481
 
Option via COS: for N is 1024 with interval = 

In [175]:
dict_prices[2200]

[[42.947175304701766,
  42.947175321475484,
  42.94717531937324,
  42.94717513102234,
  42.91072047407505,
  5.490856227111952,
  42.94718173978515,
  7495.515884386437]]

In [179]:
prices_COS_K_2200 = pd.DataFrame(dict_prices[2200], columns = test)
prices_COS_K_2200

,"[-1,1]","[-4,4]","[-8,8]","[-12,12]","[-0.5,0.5]","[-0.01,0.01]","[-16,16]","[-32,32]"
0,42.947175,42.947175,42.947175,42.947175,42.91072,5.490856,42.947182,7495.515884


In [180]:
prices_COS_K_2100 = pd.DataFrame(dict_prices[2100], columns = test)
prices_COS_K_2100

,"[-1,1]","[-4,4]","[-8,8]","[-12,12]","[-0.5,0.5]","[-0.01,0.01]","[-16,16]","[-32,32]"
0,64.83462,64.83462,64.83462,64.83462,64.735826,5.241272,64.834627,6459.307586


In [181]:
prices_COS_K_2000 = pd.DataFrame(dict_prices[2000], columns = test)
prices_COS_K_2000

,"[-1,1]","[-4,4]","[-8,8]","[-12,12]","[-0.5,0.5]","[-0.01,0.01]","[-16,16]","[-32,32]"
0,95.246692,95.246692,95.246692,95.246692,94.983962,4.991687,95.246699,2945.901781
